In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import multiprocessing
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import re
import string
from tqdm import tqdm

In [ ]:
files=['../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv',
       '../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv',
      '../input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv']

def load_data(file):
    return pd.read_csv(file)
with multiprocessing.Pool() as pool:
    test,train,sub=pool.map(load_data,files)

In [ ]:
batch_1 = train[['comment_text','target']][0:500]

In [ ]:
# remove all numbers with letters attached to them
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)

# '[%s]' % re.escape(string.punctuation),' ' - replace punctuation with white space
# .lower() - convert all strings to lowercase 
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

# Remove all '\n' in the string and replace it with a space
remove_n = lambda x: re.sub("\n", " ", x)

# Remove all non-ascii characters 
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ', x)

# Apply all the lambda functions wrote previously through .map on the comments column
batch_1['comment_text'] = batch_1['comment_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)

In [ ]:
# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, use_fast=True, model_max_length=512 )
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
from tqdm import tqdm

In [ ]:
pip install alive-progress

In [ ]:
type(batch_1['comment_text'])
batch_1['comment_text']

In [ ]:
from alive_progress import alive_bar
import time

In [ ]:
for index, value in tqdm(batch_1['comment_text'].items()):
    batch_1['comment_text'][index]=tokenizer.encode(value, add_special_tokens=True, truncation=True, max_length=512)
        
        

In [ ]:
tokenized=batch_1['comment_text']

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
np.array(padded).shape

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = batch_1['target']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
train_labels_1

In [ ]:
train_labels_1=np.where(train_labels>0.5, 1, 0)

In [ ]:
test_features

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels_1)

In [ ]:
test_labels_1=np.where(test_labels>0.5, 1, 0)

In [ ]:
lr_clf.score(test_features, test_labels_1)

In [ ]:
test

In [ ]:
#p=clf.predict_proba()[:,1]

In [ ]:
#tokenized =batch_1['comment_text'].apply(tqdm(lambda x: tokenizer.encode(x, add_special_tokens=True)))